In [1]:
from data_reader import SimulationDataset, generate_metadata
from learned_simulator import Simulator
import json
import torch
import lightning as L
from torch.utils.data import DataLoader

INPUT_SEQUENCE_LENGTH = 6
DEVICE = 'cpu'

ds = SimulationDataset(device=DEVICE, mode='test', window_size=6)
# metadata = generate_metadata(ds,mode='train')

with open('metadata.json', 'rb') as f:
    metadata = json.load(f)

num_steps = metadata['sequence_length'] - INPUT_SEQUENCE_LENGTH

ValueError: too many values to unpack (expected 3)

In [14]:
train_loader = DataLoader(ds, batch_size=2, shuffle=True, num_workers=0)

for position_sequence, next_position in train_loader:
    n_particles_per_example = [position_sequence.shape[1] for _ in range(2)]
    batch_size, num_particles, window_size, pos_dim = position_sequence.shape
    position_sequence = position_sequence.view(-1, window_size, pos_dim)  # shape: (batch_size * num_particles, window_size, pos_dim)
    next_position = next_position.view(-1, pos_dim)

    particle_types = torch.full((position_sequence.shape[0],), 5)


    noisy_position_sequence = position_sequence 
    print(torch.cat([torch.LongTensor([i for _ in range(n)]) for i, n in enumerate(n_particles_per_example)]))
    
    break

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [8]:
noisy_position_sequence.shape

torch.Size([722, 6, 2])

In [2]:
torch.cat([torch.LongTensor([i for _ in range(n)]) for i, n in enumerate(100)])

TypeError: 'int' object is not iterable

In [11]:
dataloader = DataLoader(ds, batch_size=2, shuffle=False, num_workers=0)

In [13]:
dataloader

TypeError: 'DataLoader' object is not subscriptable

In [2]:
checkpoint_path = "lightning_logs/version_16/checkpoints/epoch=0-step=10.ckpt"
simulator = Simulator.load_from_checkpoint(
    checkpoint_path,
    particle_dimension=2,
    node_in=30,
    edge_in=3,
    latent_dim=128,
    num_message_passing_steps=10,
    mlp_num_layers=2,
    mlp_hidden_dim=128,
    num_particle_types=9,
    particle_type_embedding_size=16,
    metadata=metadata,
    strategy='baseline',
    device='cpu',
    )

In [3]:
def eval_single_rollout(simulator, features, num_steps, device):
    initial_positions = features[:, 0:INPUT_SEQUENCE_LENGTH]
    ground_truth_positions = features[:, INPUT_SEQUENCE_LENGTH:]
    
    current_positions = initial_positions
    predictions = []
    for step in range(num_steps):
        next_position = simulator.predict_positions(
            current_positions,
            n_particles_per_example=features['n_particles_per_example'],
            particle_types=features['particle_type'],
        ) # (n_nodes, 2)
        # Update kinematic particles from prescribed trajectory.
        kinematic_mask = (features['particle_type'] == 3).clone().detach().to(device)
        next_position_ground_truth = ground_truth_positions[:, step]
        kinematic_mask = kinematic_mask.bool()[:, None].expand(-1, 2)
        next_position = torch.where(kinematic_mask, next_position_ground_truth, next_position)
        predictions.append(next_position)
        current_positions = torch.cat([current_positions[:, 1:], next_position[:, None, :]], dim=1)
    predictions = torch.stack(predictions) # (time, n_nodes, 2)
    ground_truth_positions = ground_truth_positions.permute(1,0,2)
    loss = (predictions - ground_truth_positions) ** 2
    output_dict = {
        'initial_positions': initial_positions.permute(1,0,2).cpu().numpy(),
        'predicted_rollout': predictions.cpu().numpy(),
        'ground_truth_rollout': ground_truth_positions.cpu().numpy(),
        'particle_types': features['particle_type'].cpu().numpy(),
    }
    return output_dict, loss

In [19]:
for batch in dataloader:
    print(batch[0][0].shape)

torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361, 6, 2])
torch.Size([361,

In [4]:
initial_positions = ds[0][0]
current_positions = initial_positions
ground_truth_rollout = []
predictions = []

for features, labels in ds:
    # particle types are not really being used, only if we had obstacles
    particle_types = torch.full((current_positions.shape[0],), 5).to(DEVICE)
    
    next_position = simulator.predict_positions(
        current_positions,
        None,
        particle_types
    )

    ground_truth_rollout.append(labels)
    predictions.append(next_position)
    current_positions = torch.cat([current_positions[:, 1:], next_position[:, None, :]], dim=1)

ground_truth_rollout = torch.stack(ground_truth_rollout)
predictions = torch.stack(predictions)

loss = (predictions - ground_truth_rollout) ** 2

output_dict = {
    'initial_positions': initial_positions.permute(1,0,2).cpu().numpy(),
    'predicted_rollout': predictions.detach().cpu().numpy(),
    'ground_truth_rollout': ground_truth_rollout.cpu().numpy(),
    'particle_types': particle_types.cpu().numpy(),
}

In [5]:
output_dict['predicted_rollout']

array([[[-2.14397341e-01,  2.73891211e-01],
        [-2.31300890e-01,  1.91038370e-01],
        [-2.93496013e-01,  2.16028988e-01],
        ...,
        [-3.56864959e-01,  7.95827061e-02],
        [-3.66994709e-01,  1.11651555e-01],
        [-4.04841244e-01,  1.17947906e-01]],

       [[-9.79604483e-01,  4.40633744e-01],
        [-1.01433802e+00,  3.42324913e-01],
        [-1.11640990e+00,  3.49762022e-01],
        ...,
        [-1.34542835e+00,  6.76926374e-02],
        [-1.35611188e+00,  1.09357968e-01],
        [-1.44290733e+00,  9.40192938e-02]],

       [[-2.03335166e+00,  7.68786788e-01],
        [-2.09021473e+00,  6.14206851e-01],
        [-2.24051046e+00,  4.96877372e-01],
        ...,
        [-2.65527773e+00,  2.06629738e-01],
        [-2.66239643e+00,  2.30335474e-01],
        [-2.71944737e+00,  2.42481574e-01]],

       ...,

       [[-3.15984656e+05,  3.85122227e+04],
        [-3.16300875e+05,  3.75309453e+04],
        [-3.16256312e+05,  3.75168516e+04],
        ...,
     